## 7.함수 데커레이터와 클로저
### 7.1 데커레이터 기본 지식
데커레이터<sup>Decorators</sup>는 다른 함수를 인수로 받는 콜러블이다. 그리고 엄밀히 말해 데커레이터는 편리한 구문일 뿐이다. 그러나 런타임에 프로그램 행위를 변경하는 메타프로그램을 할 때 상당히 편리하다. 

```
@decorate
def target():
    print('running target()')
```
위 코드는 다음 코드와 동일하게 작동한다.
```
def target():
    print('running target()')
target = decorate(target)
```

In [1]:
""" [예제 7-1] 일반적으로 데커레이터는 함수를 다른 함수로 대체한다. """
def deco(func):
    def inner():
        print('running inner()')
    return inner 

In [2]:
@deco
def target():
    print('running target') 

In [3]:
target(), target # target은 deco의 반환값인 inner를 가리키게 된다. 

running inner()


(None, <function __main__.deco.<locals>.inner()>)

### 7.2 파이썬이 데커레이터를 실행하는 시점
데이레이터의 핵심 특징은 데커레이트 된 함수가 정의된 직후에 실행된다는 것이다. 이는 일반적으로 파이썬 모듈을 로딩하는 시점, 즉 임포트 타임에 실행된다. 데커레이터 함수가 실제 코드에 흔히 사용되는 방식과 비교해서 [예제 7-2]는 다음 두 가지 차이점이 있다. 

+ 데커레이터 함수가 데커레이트되는 함수와 같음 모듈에 정의되어 있다. 일반적으로 실제 코드에서는 데커레이터를 정의하는 모듈과 데커레이터를 적용하는 모듈을 분리해서 구현한다.
+ register( ) 데커레이터가 인수로 전달된 함수와 동일한 함수를 반환한다. 실제 코드에서 대부분의 데커레이터는 내부 함수를 정의해서 반환한다.

In [4]:
""" [예제 7-2] registration.py 모듈 """
""" 함수 데커레이터는 모듈이 임포트되자마자 실행되지만, 데커레이트된 함수는 명시적으로 호출될 때만 실행됨을 알 수 있다. """

registry = []

def register(func): # 임포트 될 때 f1, f2의 register 데커레이트로 인해 2번 호출된다.
    print('running regsiter(%s)' % func)
    registry.append(func)
    return func

@register # import 시점에서 먼저 실행
def f1():
    print('running f1()')

@register # import 시점에서 먼저 실행
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

running regsiter(<function f1 at 0x7fb4b444c598>)
running regsiter(<function f2 at 0x7fb4b44051e0>)
running main()
registry -> [<function f1 at 0x7fb4b444c598>, <function f2 at 0x7fb4b44051e0>]
running f1()
running f2()
running f3()


### 7.3 데커레이터로 개선한 전략 패턴
[예제 6-6]에서 가장 큰 문제는 함수를 정의할 때, 그리고 가장 큰 할인 방식을 결정하는 best_promo() 함수에 의해서 사용되는 promos 리트스에 함수명을 반복해서 사용한다는 점이다(코드를 반복하는 것은 문제가 있다). [예제 7-3]에서는 등록 데커레이터를 이용해서 이 문제를 해결한다. 이 방법은 다음과 같은 장점을 가진다. 
<ul>
    <li> 프로모션 전략 함수명이 특별한 형태로 되어 있을 필요없다. _promo로 끝나지 않아도 된다.</li>
    <li> 임시로 어떤 프로모션을 배제할 수 있다. 단지 데커레이터만 주석처리하면 된다. </li>
    <li> 프로모션 할인 전략을 구현한 함수는 @promotion 데커레이터가 적용되는 한 어느 모듈에서든 정의할 수 있다. </li>
</ul>

In [5]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion # 단지 주석처리 만으로 best_promo 대상에서 제외할 수 있다.
def fidelity(order): # 첫 번째 전략
    """ 충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용 """
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0 # 파이썬 3항식

@promotion
def bulk_item(order): # 두 번째 전략
    """ 20개 이상의 동일 상품을 구입하면 10% 할인 적용 """
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order): # 세 번째 전략
    """ 10종류 이상의 상품을 구입하면 전체 7% 할인 적용 """
    distinct_items = {item.product for item in order.cart} # 집합을 통해 중복을 제거
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

### 7.4 변수 범위 규칙
함수 안에 전역변수 b와 동일한 이름의 지역변수 b를 할당하는 문장이 있는 경우 b를 지역변수로 할당한다. 인터프리터가 b를 전역변수로 다루기 원한다면 global 키워드를 이용해서 선언해야 한다.

In [6]:
""" [예제 7-5] 함수 본체 안에서 값을 할당하기 때문에 지역 변수가 되는 b """
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
    
print (f2(3)) # 6이 출력되지 않고 에러가 출력됨

3


UnboundLocalError: local variable 'b' referenced before assignment

In [7]:
b = 6
def f3(a):
    global b # b를 전역변수로 간주하여 에러가 생기지 않는다. 
    print(a)
    print(b)
    b = 9
    
print (f3(3))

3
6
None


In [8]:
""" 바이트 코드 비교 """
from dis import dis
print("byte code of f2")
print(dis(f2)) # load_fast는 지역변수, load_global은 전역변수를 읽는 것을 의미

print("byte code of f3")
print(dis(f3))

byte code of f2
  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE
None
byte code of f3
  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST      

### 7.5 클로저
클로저<sup>closure</sup>는 익명 함수와 다른 개념으로 함수 본체에서 정의하지 않고 참조하는 비전역<sup>nonglobal</sup> 변수를 포함한 확장 범위를 가진 함수이다. 이해하기 어려운 개념이므로 예제를 통해 살펴보자. avg( )는 매일 새로운 가격이 추가되고 지금까지의 모든 가격을 고려해서 평균을 구한다.

In [9]:
""" [예제 7-8] 이동평균을 계산하는 클래스 """
class Averager:
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
avg = Averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

In [10]:
""" [예제 7-9] 고위 함수를 사용 """
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

avg = make_averager() 
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

Averager 클래스의 avg( ) 함수가 데이터를 보관하는 방법은 명확하다. 그러나 고위 함수를 사용하는 경우 series를 어디서 찾을까? make_averager( ) 함수 본체 안에서 series=[ ]로 초기화하고 있으므로 series는 이 함수의 지역 변수다. 그렇지만 avg(10)을 호출할 때, make_averager( ) 함수는 이미 반환했으므로 지역범위도 이미 사라진 후다.

average 안에 있는 series는 자유 변수<sup>free variable</sup>다. 자유 변수라는 말은 지역 볌위에 바인딩되어 있지 않은 변수를 의미한다. 따라서 클로저는 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수다. 따라서 함수를 정의하는 범위가 사라진 후에 함수를 호출해도 자유 변수에 접근할 수 있다.

In [11]:
""" [예제 7-11] make_averager()로 생성한 함수 조사하기 """
print("local variables:", avg.__code__.co_varnames) 
print("glocal variables:", avg.__code__.co_freevars)

print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

local variables: ('new_value', 'total')
glocal variables: ('series',)
(<cell at 0x7fb4b437ab28: list object at 0x7fb4b451bac8>,)
[10, 11, 12]


### 7.6 nonlocal 선언
[예제 7-9]과 다르게 [예제 7-13]은 에러가 발생한다. 이는 count, total에 대입이 일어나는 경우 local 변수로 인식하기 때문이다(7.4 변수의 범위 참조). 때문에 count, total이 local이 아니고 free임을 [예제 7-14]와 같이 <b>nonlocal</b> 키워드를 통해 선언할 수 있다. 

In [12]:
""" [예제 7-13] 잘못된 고위 함수 """
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    
    return averager

avg = make_averager() 
avg(10), avg(11), avg(12)

UnboundLocalError: local variable 'count' referenced before assignment

In [13]:
""" [예제 7-14] nonlocal 키워드로 오류 해결- """
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

avg = make_averager() 
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

### 7.7 간단한 데커레이터 구현하기
[예제 7-15]는 데커레이트된 함수를 호출할 때마다 시간을 측정해서 실행에 소요된 시간, 전달된 인수, 반환값을 출력하는 데커레이터이다.

In [14]:
""" [예제 7-15] 함수의 실행 시간을 출력하는 간단한 데커레이터 """
import time

def clock(func):
    def clocked(*args): 
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args) # func의 argument들을 ,로 구분하여 문자를 생성
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result)) #%fs에서 s는 초단위 출력
        return result
    return clocked

In [15]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12318176s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000066s] factorial(1) -> 1
[0.00002871s] factorial(2) -> 2
[0.00005101s] factorial(3) -> 6
[0.00007151s] factorial(4) -> 24
[0.00009182s] factorial(5) -> 120
[0.00011556s] factorial(6) -> 720
6! = 720


#### 7.7.1 작동 과정
아래 두 코드는 동일하다. 
```
@clock
def factorial(n)
    return 1 if n < 2 else n*factorial(n-1)
```

```
def factorial(n)
    return 1 if n < 2 else n*factorial(n-1)
    
factorial = clock(factorial)
```

In [16]:
factorial.__name__ # 파이썬 인터프리터가 내부적으로 clocked를 factorial에 할당했다. 

'clocked'

이 예제 코드는 전형적인 데커레이터의 작동 방식을 보여준다. 데커레이트된 함수를 동일한 인수를 받는 함수로 교체하고, (일반적으로) 데커레이트된 함수가 반환해야 하는 값을 반환하면서, 추가적인 처리를 수행한다. 그러나 [예제 7-15]에서 구현한 clock( ) 데커레이터는 키워드 인수를 지원하지 않으며, 데커레이트된 함수의 \_\_name\_\_과 \_\_doc\_\_ 속성을 가린다. [예제 7-17]은 functools.wraps( ) 데커레이터를 이용해서 func에서 clocked로 관련된 속성을 복사한다. 

In [17]:
""" [예제 7-17] 개선된 clock 데커레이터 """
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args)) # func의 argument들을 ,로 구분하여 문자를 생성
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result)) #%fs에서 s는 초단위 출력
        return result
    return clocked

In [18]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n, rev=0):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6, rev=1))
    
print(factorial.__name__) # clocked가 아닌 factorial을 출력한다.

**************************************** Calling snooze(.123)
[0.12317806s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000066s] factorial(1) -> 1
[0.00005362s] factorial(2) -> 2
[0.00008087s] factorial(3) -> 6
[0.00010188s] factorial(4) -> 24
[0.00012203s] factorial(5) -> 120
[0.00014691s] factorial(6, rev=1) -> 720
6! = 720
factorial


### 7.8 표준 라이브러리에서 제공하는 데커레이터
파이썬에서는 메서드를 데커레이트하기 위해 property( ), classmethod( ), staticmethod( ) 총 3개의 내장함수를 제공한다(추후에 설명). 그리고 자주 볼 수 있는 데커레이터 중에는 functools.wraps( )가 있다. 이 함수는 제대로 작동하는 데커레이터를 만들기 위한 헬퍼로 그 중 lru_cache( ), singledispatch( )가 가장 흥미롭다.

#### 7.8.1 functools.lru_cache( )를 이용한 메모리제이션
메모이제이션은 이전에 실행한 값비싼 함수의 결과를 저장함으로써 이전에 사용된 인수에 대해 다시 계산할 필요가 없게 해준다. 이름 앞에 붙은 LRU는 'Least Recently Used'의 약자로, 오랫동안 사용하지 않은 항목을 버림으로써 캐시가 무한정 커지지 않음을 의미한다. [예제 7-18]에서 보는 것처럼 n번째 피보나치 수열을 생성하기 위해 아주 느리게 실행되는 재귀 함수에서 lru_cache()가 진가를 발휘한다.

In [19]:
""" [예제 7-18] 일반적인 피보나치 수열 계산 """

@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == "__main__":
    print(fibonacci(6))

[0.00000040s] fibonacci(0) -> 0
[0.00000078s] fibonacci(1) -> 1
[0.00019084s] fibonacci(2) -> 1
[0.00000029s] fibonacci(1) -> 1
[0.00000038s] fibonacci(0) -> 0
[0.00000032s] fibonacci(1) -> 1
[0.00003255s] fibonacci(2) -> 1
[0.00006599s] fibonacci(3) -> 2
[0.00031091s] fibonacci(4) -> 3
[0.00000023s] fibonacci(1) -> 1
[0.00000021s] fibonacci(0) -> 0
[0.00000029s] fibonacci(1) -> 1
[0.00003075s] fibonacci(2) -> 1
[0.00006040s] fibonacci(3) -> 2
[0.00000020s] fibonacci(0) -> 0
[0.00000028s] fibonacci(1) -> 1
[0.00002971s] fibonacci(2) -> 1
[0.00000023s] fibonacci(1) -> 1
[0.00000035s] fibonacci(0) -> 0
[0.00000028s] fibonacci(1) -> 1
[0.00003068s] fibonacci(2) -> 1
[0.00006034s] fibonacci(3) -> 2
[0.00011951s] fibonacci(4) -> 3
[0.00020975s] fibonacci(5) -> 5
[0.00055123s] fibonacci(6) -> 8
8


In [20]:
""" [예제 7-19] 캐시를 이용한 더 빠른 구현 """
import functools

"""
maxsize=128 : 얼마나 많은 호출을 저장할 지 결정
typed=False : True이면 인수의 자료형이 다른 경우 따로 결과를 저장,
※ 참고 : 인수는 Hash 가능해야 함
"""
@functools.lru_cache()
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__ == "__main__": 
    print(fibonacci(6)) 

[0.00000067s] fibonacci(0) -> 0
[0.00000092s] fibonacci(1) -> 1
[0.00013676s] fibonacci(2) -> 1
[0.00000155s] fibonacci(3) -> 2
[0.00045215s] fibonacci(4) -> 3
[0.00000251s] fibonacci(5) -> 5
[0.00066336s] fibonacci(6) -> 8
8


#### 7.8.2 단일 디스패치를 이용한 범용 함수
파이썬에서는 인수의 타입에 따라 호출을 달리하는 함수의 오버로딩을 지원하지 않는다. 이에 대한 대안으로 functools.singledispatch() 데커레이터를 사용할 수 있다. 일반 함수를 singledispatch()로 데커레이트하면 이 함수는 범용 함수<sup>generic function</sup>가 되며, 첫 번째 인수의 자료형에 따라 서로 다른 방식으로 연산을 수행하게 된다.

가능하면 int, list와 같은 클래스보다 numbers.Integral 이나 abc.MutableSequence와 같은 추상 베이스 클래스를 처리하도록 특화된 함수를 등록하는 것이 좋다. 추상 베이스 클래스로 등록하면 호환되는 자료형을 폭넓게 지원할 수 있다. 예를 들어 파이썬 확장은 int형의 대안으로 고정된 길이의 비트를 numbers.Intergral의 서브클래스로 제공할 수 있다.
※ signledispatch는 자바 스타일의 메서드 오버로딩을 파이썬에 적용하기 위해 설게된 것은 아니다. signledispatch는 모듈화된 학장을 지원한다. 각 모듈은 자신이 지원하는 자료형에 대한 특화된 함수를 등록할 수 있다.

In [21]:
""" [예제 7-21] 여러 함수를 범용 함수로 묶는 커스텀 htmlize.register()를 생성하는 singledispatch """
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj)) # https://docs.python.org/3/library/html.html
    return '<pre>{0}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

print(htmlize("hello world"))
print(htmlize(7))
print(htmlize((12345, 12345)))
print(htmlize(("text", "text")))
print(htmlize(1.2345))


<p>hello world</p>
<pre>7 (0x7)</pre>
<ul>
<li><pre>12345 (0x3039)</pre></li>
<li><pre>12345 (0x3039)</pre></li>
</ul>
<ul>
<li><p>text</p></li>
<li><p>text</p></li>
</ul>
<pre>1.2345</pre>


### 7.9 누적된 데커레이터
하나의 함수 f()에 두 데커레이터 @d1, @d2를 차례대로 적용하면 결과는 f=d1(d2(f))와 동일하다.
```
@d1
@d2
def f():
    print('f')
```
```
def f():
    print('f')

f = d1(d2(f))
```


### 7.10 매개변수화된 데커레이터
Register 예를 살펴보자

In [22]:
""" [예제 7-22] Registration """

registry = []
def register(func):
    print("running register {}".format(func))
    registry.append(func)
    return func

@register
def f1():
    print("runnung f1()")
    
print('runnung main()')
print('registry ->', registry)

f1()

running register <function f1 at 0x7fb4b444cb70>
runnung main()
registry -> [<function f1 at 0x7fb4b444cb70>]
runnung f1()


#### 7.10.1 매개변수화된 등록 데커레이터
register()가 등록하는 함수를 활성화 혹은 비활성화하기 쉽게 만들기 위해, 인수 active를 받도록 만들어보자.

In [23]:
registry = set() # 함수의 추가와 제거를 빠르게 하기 위해 registry를 집합형으로 정의

def register(active=True): # 선택 여부를 판단하기 위한 인수를 받는다.
    def decorate(func): # decorate() 내부 함수가 실제 데커레이터다. 함수를 인수로 받는다.
        print('running register(active={})->decorate({})'.format(active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        
        return func # decorate()는 데커레이터이므로 함수를 반환해야 한다.
    return decorate # register는 데커레이터 팩토리이므로 decorate()를 반환한다.

@register(active=False) # 앞에서 본 바와 같이 임포트 타임에 실행되므로 register는 2번 호출된다. 
def f1():
    print('running f1()')

@register() # 인수를 전달하지 않더라도 register는 여전히 함수로 호출해야하므로 @register()로 호출한다. 그러면 실제 데커레이터인 decorate()를 반환한다.
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7fb4b43ec840>)
running register(active=True)->decorate(<function f2 at 0x7fb4b44051e0>)


In [24]:
print(registry)
print(register()(f3)) # register()는 decorate()를 반환하고, 이 데커레이터가 f3()에 적용된다.

{<function f2 at 0x7fb4b44051e0>}
running register(active=True)->decorate(<function f3 at 0x7fb4b4405620>)
<function f3 at 0x7fb4b4405620>


In [25]:
print(registry)
print(register(active=False)(f2))
print(registry)

{<function f3 at 0x7fb4b4405620>, <function f2 at 0x7fb4b44051e0>}
running register(active=False)->decorate(<function f2 at 0x7fb4b44051e0>)
<function f2 at 0x7fb4b44051e0>
{<function f3 at 0x7fb4b4405620>}


#### 7.10.2 매개변수화된 clock 데커레이터
사용자가 포맷 몬자열을 전달해서 데커레이트된 함수가 출력할 문자열을 설정한다. 

In [33]:
import time 
import functools

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}, {kwargs}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        @functools.wraps(func)
        def clocked(*_args, **_kwargs):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            pairs = ['%s=%r' % (k, w) for k, w in sorted(_kwargs.items())]
            kwargs = ', '.join(pairs)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result # clocked()는 데커레이트된 함수를 대체하므로, 원래 함수가 반환하는 값을 반환해야 한다.
        return clocked     # decorate()는 clocked()를 반환한다. 
    return decorate        # clock()은 decorate()를 반환한다. 

if __name__ == '__main__':
    @clock()
    def snooze(seconds):
        time.sleep(seconds)
        
    for i in range(3):
        snooze(.123)

[0.12317204s] snooze(0.123, ) -> None
[0.12317944s] snooze(0.123, ) -> None
[0.12318063s] snooze(0.123, ) -> None


In [30]:
import time
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12317800521850586s
snooze: 0.12317991256713867s
snooze: 0.12317776679992676s
